In [57]:
from telnetlib import X3PAD
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 

from sklearn.utils import shuffle
import pickle 



In [58]:
df_2020 = pd.read_csv('merged_gw2020.csv',low_memory = False)
df_2021 = pd.read_csv('merged_gw2021.csv',low_memory = False)


In [59]:
season_2020 = pd.DataFrame({'season' : ['2019/20']*df_2020.shape[0]   })
season_2021 = pd.DataFrame({'season' : ['2020/21']*df_2021.shape[0]   })

In [60]:
df_2020 =pd.concat([df_2020, season_2020] ,axis = 'columns')
df_2021 =pd.concat([df_2021, season_2021] ,axis = 'columns')


In [61]:
df= pd.concat([df_2020 , df_2021], axis = 'index',ignore_index = True)


In [62]:
df["was_home"] = df["was_home"].astype(int)

In [63]:
df.sort_values(by=['name','season','GW'], inplace =True)
df.reset_index(inplace = True)


df.drop(['index'],axis= 'columns' , inplace =True)

In [64]:
df['id'] = df.groupby(['name']).ngroup()

In [65]:
df['pos']= df.groupby(['position']).ngroup()

In [66]:
df['teams'] = df.groupby(['team']).ngroup()

In [67]:
identity = df['id'].unique()

### Enter the gameweek 


In [68]:
x = 35

In [69]:

df_id = df.loc[: , ['id', 'name' ,'position' ,'team']]
df_id.reset_index(inplace = True)
df_id.drop_duplicates(subset='id',keep = 'first',inplace = True)

df_id.drop(['index'],axis= 'columns' , inplace =True)
df_id

,id,name,position,team
0,0,Aaron Connolly,FWD,Brighton
73,1,Aaron Cresswell,DEF,West Ham
146,2,Aaron Lennon,MID,Burnley
178,3,Aaron Mooy,MID,Brighton
216,4,Aaron Ramsdale,GK,Sheffield Utd
...,...,...,...,...
46799,978,Zak Swanson,DEF,Arsenal
46811,979,Zeze Steven Sessegnon,DEF,Fulham
46849,980,Álvaro Fernández,GK,Brentford
46886,981,Çaglar Söyüncü,DEF,Leicester


In [70]:
y=x
Gameweeks=[]

In [71]:
for i in range(5):
    Gameweeks.append(y)
    y=y-1


In [72]:
dataframes = {}

In [73]:
for GW in Gameweeks:
    filt=(df['GW'] == GW) & (df['season'] == '2020/21')
    dataframes[GW] = df[filt]
    dataframes[GW].reset_index(inplace = True)
    dataframes[GW].drop(['index'],axis= 'columns' , inplace =True)
    dataframes[GW].drop([ 'kickoff_time','season', 'name' , 'position','pos' ,'GW' ,'team','teams','round','element'],axis= 'columns' , inplace =True)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [74]:
n = 0
for keys in dataframes:
    column = []
    
    for value in dataframes[keys].columns:
        
        if (value != 'id')  & (value!='team'):
            value = value +f'_{n}' 
        column.append(value)
        
    dataframes[keys].columns = column
    n=n-1   

In [75]:
for keys in dataframes:
    dataframes[keys].drop_duplicates(subset='id' , keep ='first' , inplace = True)
    dataframes[keys].reset_index(inplace = True)
    dataframes[keys].drop(['index'],axis= 'columns' , inplace =True)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [76]:
df1 = pd.merge(dataframes[x] , dataframes[x-1], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-2], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-3], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-4], on = 'id' , how = 'outer')

In [77]:
filt = (df['GW'] == x) | (df['GW'] == x-1) | (df['GW'] == x-2) | (df['GW'] == x-3) | (df['GW'] == x-4)
df_position = df.loc[filt , ['id' , 'pos','teams']]

In [78]:
df_position.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_position.reset_index(inplace = True)
df_position.drop(['index'],axis= 'columns' , inplace =True)

In [79]:
df1.drop_duplicates(subset='id',keep = 'first',inplace = True)
df1.reset_index(inplace = True)
df1.drop(['index'],axis= 'columns' , inplace =True)

In [80]:
df1 = pd.merge(df1 , df_position, on = 'id' , how = 'inner')

In [81]:
columns = df1.columns
remove = ('pos','teams','id','total_points')
a = set(columns)
b = a.difference(remove)
columns = list(b)

In [82]:
df1 = df1.fillna(df1.median())

In [83]:
df1.loc[:,columns] = StandardScaler().fit_transform(df1.loc[:, columns])

In [84]:
X=df1.drop(['id'  ,'pos','teams'],axis ='columns') 


In [85]:
X.columns

Index(['xP_0', 'assists_0', 'bonus_0', 'bps_0', 'clean_sheets_0',
       'creativity_0', 'fixture_0', 'goals_conceded_0', 'goals_scored_0',
       'ict_index_0',
       ...
       'team_a_score_-4', 'team_h_score_-4', 'threat_-4', 'total_points_-4',
       'transfers_balance_-4', 'transfers_in_-4', 'transfers_out_-4',
       'value_-4', 'was_home_-4', 'yellow_cards_-4'],
      dtype='object', length=145)

In [86]:
import pickle

with open('model_pickle','rb') as file:
    mp = pickle.load(file)

In [87]:
df1['predict'] = mp.predict(X)




In [88]:
df1['predict']

0      3.280000
1      3.190000
2      2.006667
3      3.260000
4      2.320000
         ...   
723    0.590000
724    3.160000
725    0.790000
726    1.820000
727    1.360417
Name: predict, Length: 728, dtype: float64

In [89]:
df1 = pd.merge(df1 , df_id, on = 'id' , how = 'inner')

In [90]:
df_op = df1.loc[:,['predict' , 'name' , 'position','team']]

In [91]:
df_op.isnull().values.any()

False

In [92]:
df_op

,predict,name,position,team
0,3.280000,Aaron Connolly,FWD,Brighton
1,3.190000,Aaron Cresswell,DEF,West Ham
2,2.006667,Aaron Lennon,MID,Burnley
3,3.260000,Aaron Ramsdale,GK,Sheffield Utd
4,2.320000,Aaron Ramsey,MID,Aston Villa
...,...,...,...,...
723,0.590000,Zach Awe,DEF,Arsenal
724,3.160000,Zack Steffen,GK,Man City
725,0.790000,Zak Swanson,DEF,Arsenal
726,1.820000,Álvaro Fernández,GK,Brentford


In [93]:
df_op.to_csv('final_result.csv')